In [122]:
import pandas as pd
data = pd.read_csv('data/new_with_street.csv')
data = data.dropna()
data = data.reset_index()
data.drop(['index'], axis=1, inplace=True)
start_date = '2018-12-01'
end_date = '2019-12-01'
mask = (data['date'] >= start_date) & (data['date'] <= end_date)
df = data.loc[mask]

In [123]:
df.shape

(11294, 4)

In [37]:
df = df[:20]

In [51]:
import requests
import json

In [98]:
res = requests.get(f'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={api}&geocode={addr}')

In [97]:
addr

'Санкт-Петербург, Мурманским Ленинградским'

In [101]:
loc = json.loads(res.content.decode('utf8'))['response']['GeoObjectCollection']['featureMember']['GeoObject']['Point']['pos'].split(' ')
location = [float(loc[1]), float(loc[0])]

TypeError: list indices must be integers or slices, not str

In [100]:
location

[59.939095, 30.315868]

In [121]:
from folium.plugins import MarkerCluster
import folium
from geopy.geocoders import Yandex
from tqdm.autonotebook import tqdm, trange
f = open('api_key.txt', 'r')
api = f.read()
f.close()
geolocator = Yandex(api)
ad = []
spb = [59.939095, 30.315868]
map = folium.Map(location=spb, zoom_start = 10)
marker_cluster = MarkerCluster().add_to(map)
for i in tqdm(list(df.index.values)):
    st = df['pavlov'][i]
    if st != '':
        id = df['id'][i]
        addr = f'Санкт-Петербург, {st}'
        try:
            res = requests.get(f'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={api}&geocode={addr}')
            loc = json.loads(res.content.decode('utf8'))['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(' ')
            location = [float(loc[1]), float(loc[0])]
        except:
            location = None
        if location is None:
            continue
        if round(location[0], 6) == spb[0] and round(location[1], 6) == spb[1]:
            continue
        folium.Marker(location=[location[0], location[1]], popup = folium.Popup(f'<a href="https://vk.com/spb_today?w=wall-68471405_{id}"target="_blank">link to news in VK</a>'), icon=folium.Icon(color = 'red', icon='info-sign')).add_to(marker_cluster)
map.save("map1.html")

In [120]:
st = df['pavlov'][i]
id = df['id'][i]
addr = f'Санкт-Петербург, {st}'
res = requests.get(f'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={api}&geocode={addr}')
loc = json.loads(res.content.decode('utf8'))['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(' ')
location = [float(loc[1]), float(loc[0])]